In [1]:
from __future__ import annotations

from xdsl.dialects.builtin import ModuleOp

from toy_to_riscv.helpers import (parse_toy, print_module, optimise_toy, lower_toy, emulate_riscv)

from riscv.emulator_iop import run_riscv, print_riscv_ssa

import toy.dialect as td
import riscv.riscv_ssa as rd
import toy_to_riscv.dialect as trd

### WIP

example = """
def main() {
  var a<2, 3> = [[1, 2, 3], [4, 5, 6]];
  var b<6> = [1, 2, 3, 4, 5, 6];
  var c<2, 3> = b;
  var d = a + c;
  print(d);
}
"""

### WIP

toy_0 = parse_toy(example)
print_module(toy_0)
print()


"builtin.module"() ({
  "toy.func"() ({
    %0 = "toy.constant"() {"value" = dense<[[1, 2, 3], [4, 5, 6]]> : tensor<2x3xi32>} : () -> tensor<2x3xi32>
    %1 = "toy.reshape"(%0) : (tensor<2x3xi32>) -> tensor<2x3xi32>
    %2 = "toy.constant"() {"value" = dense<[1, 2, 3, 4, 5, 6]> : tensor<6xi32>} : () -> tensor<6xi32>
    %3 = "toy.reshape"(%2) : (tensor<6xi32>) -> tensor<6xi32>
    %4 = "toy.reshape"(%3) : (tensor<6xi32>) -> tensor<2x3xi32>
    %5 = "toy.add"(%1, %4) : (tensor<2x3xi32>, tensor<2x3xi32>) -> tensor<2x3xi32>
    "toy.print"(%5) : (tensor<2x3xi32>) -> ()
    "toy.return"() : () -> ()
  }) {"sym_name" = "main", "function_type" = () -> ()} : () -> ()
}) : () -> ()



In [2]:
toy_1 = optimise_toy(toy_0)
print_module(toy_1)
print()

"builtin.module"() ({
  "toy.func"() ({
    %0 = "toy.constant"() {"value" = dense<[[1, 2, 3], [4, 5, 6]]> : tensor<2x3xi32>} : () -> tensor<2x3xi32>
    %1 = "toy.constant"() {"value" = dense<[[1, 2, 3], [4, 5, 6]]> : tensor<2x3xi32>} : () -> tensor<2x3xi32>
    %2 = "toy.add"(%0, %1) : (tensor<2x3xi32>, tensor<2x3xi32>) -> tensor<2x3xi32>
    "toy.print"(%2) : (tensor<2x3xi32>) -> ()
    "toy.return"() : () -> ()
  }) {"sym_name" = "main", "function_type" = () -> ()} : () -> ()
}) : () -> ()



In [3]:
riscv_0 = lower_toy(toy_1)
print_module(riscv_0)
print()

### WIP

code = print_riscv_ssa(riscv_0)
print(code)
print()

### WIP

emulate_riscv(code)

"builtin.module"() ({
  "riscv.section"() ({
    "riscv_ssa.label"() {"label" = #riscv.label<heap>} : () -> ()
    "riscv_ssa.directive"() {"directive" = ".space", "value" = "1024"} : () -> ()
  }) {"directive" = ".bss"} : () -> ()
  "riscv.section"() ({
    "riscv_ssa.label"() {"label" = #riscv.label<main.tensor_shape.0>} : () -> ()
    "riscv_ssa.directive"() {"directive" = ".word", "value" = "0x2, 0x2, 0x3"} : () -> ()
    "riscv_ssa.label"() {"label" = #riscv.label<main.tensor_data.0>} : () -> ()
    "riscv_ssa.directive"() {"directive" = ".word", "value" = "0x6, 0x1, 0x2, 0x3, 0x4, 0x5, 0x6"} : () -> ()
    "riscv_ssa.label"() {"label" = #riscv.label<main.tensor_shape.1>} : () -> ()
    "riscv_ssa.directive"() {"directive" = ".word", "value" = "0x2, 0x2, 0x3"} : () -> ()
    "riscv_ssa.label"() {"label" = #riscv.label<main.tensor_data.1>} : () -> ()
    "riscv_ssa.directive"() {"directive" = ".word", "value" = "0x6, 0x1, 0x2, 0x3, 0x4, 0x5, 0x6"} : () -> ()
  }) {"directive" = ".d